In [16]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from model_config import ModelConfig
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
transition_window =10

warmup_start = '20210421'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

# Learning rate
learning_rate = 0.1

In [18]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)
vax_statuses = [Vax.yes, Vax.no]
x_train = tf.cast(df.loc[train_start:train_end,'Rt'].values, dtype=tf.float32)
x_test = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)


y_train = {}
y_train['G_in'] = tf.cast(df.loc[train_start:train_end,'general_ward_in'], dtype=tf.float32)
y_train['G_count'] = tf.cast(df.loc[train_start:train_end,'general_ward_count'], dtype=tf.float32)
y_train['I_count'] = tf.cast(df.loc[train_start:train_end,'icu_count'], dtype=tf.float32)
y_train['D_in'] = tf.cast(df.loc[train_start:train_end,'deaths_covid'], dtype=tf.float32) + 1

y_test = {}
y_test['G_in'] = tf.cast(df.loc[train_start:test_end,'general_ward_in'], dtype=tf.float32)
y_test['G_count'] = tf.cast(df.loc[train_start:test_end,'general_ward_count'], dtype=tf.float32)
y_test['I_count'] = tf.cast(df.loc[train_start:test_end,'icu_count'], dtype=tf.float32)
y_test['D_in'] = tf.cast(df.loc[train_start:test_end,'deaths_covid'], dtype=tf.float32)

In [19]:
config = ModelConfig.from_json('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/model_config.json')

In [20]:
model = CovidModel([Vax.no, Vax.yes], [Comp.A, Comp.M, Comp.G, Comp.GR, Comp.I, Comp.IR, Comp.D],
                 transition_window,
                config, posterior_samples=1000, debug_disable_theta=False)

pre_training_preds=model.call(x_train)

In [21]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate,#beta_1=0.1, beta_2=0.1
)
#loss((y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in']), pre_training_preds)

In [22]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/linear_warm_fixed_loss_2')

In [23]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([(y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in'])]),
         epochs=2000, batch_size=0,
        callbacks=logging_callbacks)

Epoch 1/2000
G count: nan
G in: nan
I count: nan
D in: nan


2022-02-27 22:51:17.057918: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


1/1 [==============================] - 47s 47s/step - loss: nan
Epoch 2/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 47s 47s/step - loss: nan
Epoch 3/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 45s 45s/step - loss: nan
Epoch 4/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 44s 44s/step - loss: nan
Epoch 5/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 44s 44s/step - loss: nan
Epoch 6/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 44s 44s/step - loss: nan
Epoch 7/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 44s 44s/step - loss: nan
Epoch 8/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [==============================] - 44s 44s/step - loss: nan
Epoch 9/2000
G count: nan
G in: nan
I count: nan
D in: nan
1/1 [===========

KeyboardInterrupt: 

In [1]:
model.variables

NameError: name 'model' is not defined

In [ ]:
config.rho_M.value

In [15]:
isinstance(model.unconstrained_rho_M[0]['loc'], tf.Variable)

True

In [ ]:
model._constrain_parameters()

In [ ]:
model._sample_and_reparameterize()

In [ ]:
model.rho_M_params

In [ ]:
model.rho_M_samples_constrained

In [ ]:
model.warmup_A_params